# Project 2 - Diagnosis coding from clinical summary using prompt engineering



---


#####################################################################################

**[Double-click to write down your zID, Name and Surname]**

Write your zID below and replace the placeholder “zID”, "Name" and "FamilyName" in the Jupyter Notebook name with your actual zID, name and family name. That is, instead of `zID-Project-02-Name-FamilyName.ipynb`, write your actual zID, Name and Family Name.

**zID:** z5567497

**Name:** Ilia

**Surname:** Kuznetsov

**Honour Pledge** <p>
    
Declaration: <p>
    
    
I declare that this assessment item is **my own work**, except where acknowledged, and has not been submitted for academic credit elsewhere or previously, or produced independently of this course (e.g. for a third party such as your place of employment) and acknowledge that the assessor of this item may, for the purpose of assessing this item:

1. Reproduce this assessment item and provide a copy to another member of the University; and/or

2. Communicate a copy of this assessment item to a plagiarism checking service (which may then retain a copy of the assessment item on its database for the purpose of future plagiarism checking).

**By writing your zID, name and surname above, you certify that you have read and agreed to the honour pledge.**

#####################################################################################



---



---



---



Please write the questions as they are written in the [Word document](https://unsw-my.sharepoint.com/:w:/g/personal/z3368601_ad_unsw_edu_au/ERY2EhDgUPxAjrX0UFC_dHsBcc1e3hVtIxeWSIu-PJp5OA?e=iiAnOh) and your answers here. Create as many text or codes cells as needed.

## Model Loading

Run the cell below to download the `Qwen/Qwen2.5-0.5B-Instruct` model from huggingface, and running it on CPU.

DO NOT modify this cell.



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    dtype="auto",
    device_map="cpu" # Using CPU
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

## Dataset creation instruction

**Development set `patient_id`: 825, 1411, 4399, 4644, 5353**

**Test set ``patient_id``: 418, 608, 2678, 3824, 3972, 4046, 4175, 4679, 4758, 5545**

**Ground truth for the development set:**



*   Patient_id: 825. ICD1-10-AM: G05
*   Patient_id: 1411. ICD1-10-AM: A52
*   Patient_id: 4399. ICD1-10-AM: K75
*   Patient_id: 4644. ICD1-10-AM: S83
*   Patient_id: 5353. ICD1-10-AM: L03



## Evaluation Data Frame

The code in the following cell creates the pandas DataFrame containing the ground-truth labels (ICD-10-AM principal diagnosis codes) for evaluation in Question 4.

DO NOT modify this cell; Run it to create the `Eval` DataFrame for Question 4.

In [ ]:
import pandas as pd

data = [
    (418,  "C79"),
    (608,  "M85"),
    (2678, "C49"),
    (3824, "I97"),
    (3972, "A18"),
    (4046, "N49"),
    (4175, "K25"),
    (4679, "D16"),
    (4758, "J85"),
    (5545, "K42"),
]


Eval = pd.DataFrame(data, columns=[
    "patient_id",
    "ICD-10-AM principal code"
])


Eval["Model generated ICD-10-AM Code"] = ""

# Question 1 – Understanding ICD-10-AM Coding and Developing Your Research

- What is ICD-10-AM diagnostic coding, and what is ICD-10-AM? Write up to 300 words.

- What is the principal diagnosis code (in the context of ICD-10-AM)? Write up to 100 words. You may refer to the following resource for guidance: https://ar-drg.laneprint.com.au/wp-content/uploads/2020/10/ACS-Sample.pdf

- What is your research question for this project? Write up to 50 words

- What is ICD-10-AM diagnostic coding, and what is ICD-10-AM?  
ICD-10-AM (International Statistical Classification of Diseases and Related Health Problems, Tenth Revision, Australian Modification) is the Australian national standard for classifying diagnoses and health conditions recorded during episodes of patient care. It is an adaptation of the World Health Organisation’s ICD-10 system, expanded and modified to meet the administrative needs of the Australian healthcare system. ICD-10-AM incorporates Australian Coding Standards, providing a framework for consistent clinical documentation, morbidity reporting, hospital funding, and health service evaluation.  
Diagnostic coding using ICD-10-AM involves translating clinicians’ notes describing medical history, symptoms, and investigations into standardised alphanumeric codes. These codes represent diseases, disorders, and injuries. Clinical coders assign the appropriate codes by reviewing the entire medical record, applying the Australian Coding Standards, and selecting the code that best reflects the patient’s diagnoses and conditions treated or investigated.  
Accurate ICD-10-AM diagnostic codes support hospital activity-based funding, health statistics for research, and ensure comparability of clinical data across time and settings. Uniform coding also helps monitor disease trends, evaluate healthcare outcomes, and guide resource allocation.  

*I used GenAI to help brainstorm ideas about the broader purposes of clinical coding beyond hospital funding and statistics*


- What is the principal diagnosis code (in the context of ICD-10-AM)?  
In ICD-10-AM, the principal diagnosis is the condition that, after the whole record has been studied, is considered responsible for the patient’s admission. The condition established after study may or may not confirm the admitting diagnosis. It is chosen based on the circumstances of care, not just the first condition listed. As an example from the Australian Coding Standards illustrates, when a patient with diabetes and coronary artery disease is admitted with severe chest pain and found to have a myocardial infarction, the myocardial infarction is coded as the principal diagnosis because it led to the hospitalisation.

- What is your research question for this project?  
 “How does the choice between different prompt strategies (one-shot prompting, few-shot prompting, and chain-of-thought prompts) influence the accuracy of tested LLM (Qwen2.5-0.5B-Instruct) in predicting ICD-10-AM principal diagnosis codes from hospital-course summaries in the Asclepius Synthetic Clinical Notes dataset?”

# Question 2 – Understanding the Asclepius dataset:

Before prompting any LLMs, it is essential to understand the data you are working with. Conduct background research on the Asclepius dataset;   
Your answer should include, but is not limited to, the following points:

- What is the original source?

- How did the author create this dataset from its source?

- What is the overall structure of the dataset?

- What is the purpose of this dataset?

*I used GenAI to create a part of the answer. I asked it to create a short version of the author's thought process from the paper to include in the answer. Still, I modified the generated answer to my style and liking; however, it saved me some time because it explained the whole process in a couple of sentences. I provided AI with text from section 2.1 of the paper and asked it to summarise and shorten it.*

The Asclepius dataset was created from **publicly available case reports** in the PMC-Patients collection (PubMed). Authors explain that one of their motivations was to avoid the restrictions of real clinical notes due to privacy risks. While datasets like MIMIC-IV exist, access to them and even products derived from them is “**only limited to credentialed individuals, such as those who have completed CITI training**".
To overcome these limitations, the authors turned to case reports as an open and privacy-safe alternative. However, case reports differ from hospital notes: they are written as polished academic narratives, whereas clinical notes are semi-structured, use abbreviations, and often contain non-standard language and grammatical errors. To make the data usable for training a clinical LLM, the **authors used GPT-3.5 to convert case reports into synthetic discharge summaries** that mimic the style and structure of real EHR documents. To avoid the model's hallucination, they added safeguards to prevent the introduction of new clinical entities. Throughout the process, clinicians reviewed samples to ensure the rewritten notes remained accurate and didn’t introduce any new medical details.  

To finetune an LLM capable of performing various clinical NLP tasks, an **instruction–answer pair dataset** is necessary. To build this dataset, the defined eight key clinical NLP tasks were used, and five clinician-verified example questions were created for each. These examples served as seeds for GPT-3.5-turbo, which was given a synthetic clinical note and asked to generate new task-specific instructions. After that, the model was prompted again – this time with both the instruction and the note - to produce the corresponding answer. This pipeline resulted in more than **158,000 instruction–answer pairs** grounded entirely in the synthetic notes.  

By doing so, authors bypass regulatory constraints and make the dataset available to the broader research community.


In [ ]:
import re
from datasets import load_dataset
from pprint import pprint
ds = load_dataset("starmpcc/Asclepius-Synthetic-Clinical-Notes")
print(ds)


DatasetDict({
    train: Dataset({
        features: ['patient_id', 'note', 'question', 'answer', 'task'],
        num_rows: 158114
    })
})


In [ ]:
# demonstraton of dataset content
example = ds["train"][0]
pprint(example)

{'answer': 'The healthcare team used a gradual approach to changing the '
           "patient's position to avoid worsening of the respiratory status "
           'and prevent respiratory failure.',
 'note': 'Discharge Summary:\n'
         '\n'
         'Patient: 60-year-old male with moderate ARDS from COVID-19\n'
         '\n'
         'Hospital Course:\n'
         '\n'
         'The patient was admitted to the hospital with symptoms of fever, dry '
         'cough, and dyspnea. During physical therapy on the acute ward, the '
         'patient experienced coughing attacks that induced oxygen '
         'desaturation and dyspnea with any change of position or deep '
         'breathing. To avoid rapid deterioration and respiratory failure, a '
         'step-by-step approach was used for position changes. The breathing '
         'exercises were adapted to avoid prolonged coughing and oxygen '
         'desaturation, and with close monitoring, the patient managed to '
         'perfo

In [ ]:
# Define the patient IDs for development and test sets
dev_patient_ids = [825, 1411, 4399, 4644, 5353]
test_patient_ids = [418, 608, 2678, 3824, 3972, 4046, 4175, 4679, 4758, 5545]

# I used GenAI to help with this code snippet.
# Filter the dataset to create development set
# https://huggingface.co/docs/datasets/v1.4.0/package_reference/main_classes.html#datasets.Dataset.filter
# Provides filter function from datasets library with a lambda function
dev_set = ds['train'].filter(lambda x: x['patient_id'] in dev_patient_ids)
test_set = ds['train'].filter(lambda x: x['patient_id'] in test_patient_ids)

print(f"Development set size: {len(dev_set)}")
print(f"Test set size: {len(test_set)}")

# Verify the patient IDs in each set
dev_ids_found = sorted(list(set(dev_set['patient_id'])))
test_ids_found = sorted(list(set(test_set['patient_id'])))

print(f"\nDevelopment set patient IDs found: {dev_ids_found}")
print(f"Expected development IDs: {sorted(dev_patient_ids)}")

print(f"\nTest set patient IDs found: {test_ids_found}")
print(f"Expected test IDs: {sorted(test_patient_ids)}")

Filter:   0%|          | 0/158114 [00:00<?, ? examples/s]

Filter:   0%|          | 0/158114 [00:00<?, ? examples/s]

Development set size: 5
Test set size: 10

Development set patient IDs found: [825, 1411, 4399, 4644, 5353]
Expected development IDs: [825, 1411, 4399, 4644, 5353]

Test set patient IDs found: [418, 608, 2678, 3824, 3972, 4046, 4175, 4679, 4758, 5545]
Expected test IDs: [418, 608, 2678, 3824, 3972, 4046, 4175, 4679, 4758, 5545]


# Question 3 – LLM prompt engineering

Your task is to create and test a range of prompts for running inference on the designated LLM (Qwen/Qwen2.5-0.5B-Instruct) in a Google Colab environment.

§ Use the hospital course summary as the input to your prompt.

§ Your goal is to generate the corresponding principal diagnosis ICD-10-AM code.

§ Your prompt must produce exactly one ICD-10-AM code in the model’s response for each admission.

§ You must output only the ICD-10-AM category, i.e., the first three characters of the code (one letter followed by two numbers).

§ You should experiment with several prompt-engineering techniques introduced in class and explore different inference hyperparameters.

§ For this question, you must use only the development set when designing and refining your prompts (similar to working with a combined training–validation set).

You will not be assessed on ICD-10-AM coding accuracy, but rather on the quality of your prompt-development process and your understanding of LLM inference.

Keep a record of all prompts you tried as evidence of your prompt-development process. Select the five that best represent your approach and rationale, and include these in Question 3 of your Jupyter Notebook. Any additional prompts should be placed in an appendix at the end of the notebook, accompanied by comments and text cells explaining them.

In [ ]:
def run_prompt(prompt):
    # 1. Chat messages
    messages = [{"role": "user", "content": prompt}]

    # 2. Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    # 3. Tokenize
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 4. Generate
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=8, # enough for something like "I21"
        do_sample=False, # greedy decoding (choose highest-probability token each step);
        temperature=0.0 # temperature is irrelevant when sampling is disabled
    )

    # 5. Extract only generated tokens
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # 6. Decode raw output
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    # 7. Extract ICD-10-AM category (Letter + 2 digits)
    match = re.search(r"[A-Z][0-9]{2}", content.upper())
    if match:
        icd10_cat = match.group(0)
    else:
        icd10_cat = "UNK"

    return content, icd10_cat

In [ ]:
zero_shot_prompt = """
    You are an expert Australian clinical coder.

    From the hospital course summary below, identify the PRINCIPAL DIAGNOSIS and output its ICD-10-AM CATEGORY code.

    Important:
    - Output EXACTLY ONE code.
    - Output ONLY the ICD-10-AM CATEGORY (first three characters: one letter followed by two digits).
    - Do NOT output any extra words, punctuation, or explanation.

    Hospital course summary:
    \"\"\"{note_text}\"\"\"

    Now output ONLY the ICD-10-AM CATEGORY code:
    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Entry 0:
raw model output: B85.1
ICD-10-AM category: B85
Entry 1:
raw model output: B86.90
ICD-10-AM category: B86
Entry 2:
raw model output: C04.1
ICD-10-AM category: C04
Entry 3:
raw model output: A87.5
ICD-10-AM category: A87
Entry 4:
raw model output: C078.1
ICD-10-AM category: C07


In [ ]:
one_shot_prompt = """
    You are an expert Australian clinical coder.

    Example:
    Hospital course summary:
    "Patient admitted with chest pain; after study diagnosed with acute myocardial infarction."
    ICD-10-AM code:
    I21

    Now code this case:
    "{note_text}"

    Rules:
    - Output EXACTLY ONE ICD-10-AM CATEGORY (1 letter + 2 digits).
    - No words, no punctuation, no explanations.
    - If unclear, output: I don't know

    Code:

"""

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = one_shot_prompt.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I2108.1
ICD-10-AM category: I21
Entry 1:
raw model output: I2100
ICD-10-AM category: I21
Entry 2:
raw model output: I2100
ICD-10-AM category: I21
Entry 3:
raw model output: I210500
ICD-10-AM category: I21
Entry 4:
raw model output: I2100
ICD-10-AM category: I21


In [ ]:
three_shot_prompt = """
    You are an expert Australian clinical coder.

Examples:

1)
Summary: "Chest pain; after study = acute myocardial infarction."
Code: I21

2)
Summary: "Fever, cough, hypoxia; CXR = pneumonia."
Code: J18

3)
Summary: "Polyuria, polydipsia, high glucose; dx = type 2 diabetes."
Code: E11

Now code this case:
{note_text}

Rules:
- Output ONE ICD-10-AM CATEGORY (1 letter + 2 digits)
- No words, no punctuation, no explanations
- If unclear, output: I don't know

Code:

"""

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = three_shot_prompt.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I21
ICD-10-AM category: I21
Entry 1:
raw model output: I21
ICD-10-AM category: I21
Entry 2:
raw model output: I21
ICD-10-AM category: I21
Entry 3:
raw model output: I21
ICD-10-AM category: I21
Entry 4:
raw model output: I21
ICD-10-AM category: I21


In [ ]:
self_check_prompt = """
    You are an expert Australian clinical coder.

    Hospital course summary:
    {note_text}

    First, silently check:
    - What condition was chiefly responsible for the admission?
    - Is it explicitly documented?
    - Does it correspond to exactly one ICD-10-AM CATEGORY code?

    Now output ONLY the 3-character ICD-10-AM code (1 letter + 2 digits), with no explanation:
    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = self_check_prompt.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: MENINGITIS
ICD-10-AM category: UNK
Entry 1:
raw model output: A 71-year-old man with
ICD-10-AM category: UNK
Entry 2:
raw model output: 111111
ICD-10-AM category: UNK
Entry 3:
raw model output: A84.50
ICD-10-AM category: A84
Entry 4:
raw model output: C078
ICD-10-AM category: C07


In [ ]:
cot_hidden_prompt = """
    You are an expert Australian clinical coder.

    Hospital course summary:
    {note_text}

    Let’s think step by step about which condition is the PRINCIPAL DIAGNOSIS and what its ICD-10-AM CATEGORY code is.

    After thinking, output ONLY the 3-character ICD-10-AM CATEGORY code (1 letter + 2 digits), with no words, no punctuation, no explanation.
"""

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = cot_hidden_prompt.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: MENINGITIS
ICD-10-AM category: UNK
Entry 1:
raw model output: SPHONYCITIS
ICD-10-AM category: UNK
Entry 2:
raw model output: CPR
ICD-10-AM category: UNK
Entry 3:
raw model output: A
ICD-10-AM category: UNK
Entry 4:
raw model output: C078
ICD-10-AM category: C07


# Question 4 – Prompt evaluation and conclusion



# Appendix A.

It is a full version of my prompt-engineering thought process.

The zero-shot prompt was designed to elicit precise ICD-10-AM category-level coding from a language model without providing any prior examples. The prompt establishes a specific expert persona (“an expert Australian clinical coder”) to orient the model toward the expected domain reasoning. The task is to identify the principal diagnosis based solely on the provided hospital course summary.
To minimise variability in output formatting, the prompt includes strict formatting constraints. The model is instructed to output exactly one ICD-10-AM code, explicitly defined as a one-letter code followed by two digits. The prompt prohibits any explanatory text, punctuation, additional diagnoses, or alternative formats, reducing the likelihood of hallucinated or extraneous content. The final instruction appears at the end of the prompt to exploit the recency effect.


In [ ]:
zero_shot_prompt = """
    You are an expert Australian clinical coder.

    From the hospital course summary below, identify the PRINCIPAL DIAGNOSIS and output its ICD-10-AM CATEGORY code.

    Important:
    - Output EXACTLY ONE code.
    - Output ONLY the ICD-10-AM CATEGORY (first three characters: one letter followed by two digits).
    - Do NOT output any extra words, punctuation, or explanation.

    Hospital course summary:
    \"\"\"{note_text}\"\"\"

    Now output ONLY the ICD-10-AM CATEGORY code:
    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    pprint(note_text)
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)


Entry 0:
('Hospital Course:\n'
 'The patient, a 15-month-old girl, was admitted to Peking Union Medical '
 'College Hospital (PUMCH) with high fever, irritability, and refusal to walk. '
 'She was initially seen at a local clinic for fever and constipation but was '
 "treated with ibuprofen. Later, she presented to Haikou People's Hospital "
 'with persistent high fever and a lumbar puncture was performed revealing an '
 'opening pressure of 140 mm H2O and clear CSF with 120 × 106/L white blood '
 'cells. The patient was treated for viral meningitis with an antiviral for 2 '
 'weeks but her fever persisted and she refused to walk. \n'
 '\n'
 'Clinical Findings:\n'
 'On physical examination, the patient had a weight of 11.5 kg and a '
 'temperature of 40°C. Rashes, lymphadenectasis, and joint redness were not '
 'observed. Skin sensation could not be evaluated because the patient '
 "responded to any skin contact with exaggeration and crying. The patient's "
 'muscle strength and tone w

Case #1:  
Clinical summary (generated by ChatGPT 5.1):  
A 15-month-old girl presented with persistent high fever, irritability, and refusal to walk. CSF and blood tests showed marked eosinophilia, and the child improved rapidly with antiparasitic and steroid treatment. This clinical picture is consistent with eosinophilic meningitis.  

Correct ICD-10-AM category:  
G05 - Encephalitis, myelitis and encephalomyelitis in diseases classified elsewhere.  

Model output:  
The model produced “B85”corresponding to a Pediculosis and phthiriasis - an incorrect classification.  

Case #2:  
Clinical summary:  
A 71-year-old man presented with progressive right hip pain, shortening of the limb, and inability to walk. Imaging and synovial biopsy revealed characteristic vascular and inflammatory changes consistent with syphilitic arthritis, and symptoms resolved after surgical repair and antibiotic therapy.  

Correct ICD-10-AM category:  
 A52 – Late syphilis.  

Model output:  
B86 - Scabies (Incorrect).


Case #3:   
Clinical summary:  
A 41-year-old man presented with fever, anorexia, nausea, and abnormal liver function tests. Imaging revealed portal vein thrombosis and inflammatory changes, and blood cultures grew Streptococcus anginosus. Further investigations identified a cholecysto-colonic fistula. He was diagnosed with thrombophlebitis of the portal vein with associated hepatobiliary inflammation, treated with antibiotics, anticoagulation, and later cholecystectomy with partial colectomy.  

Correct ICD-10-AM category:  
K75 - Other inflammatory liver diseases.  

Model output:  
The model generated “C04.1”, reduced to C04, corresponding to a malignant neoplasm of the floor of the mouth, which is clinically implausible for this case.  

Case #4:  
Clinical summary:  
A 23-year-old man presented with knee pain and limited motion after a football injury. Imaging confirmed a lateral patellar dislocation, and unsuccessful closed reduction required arthroscopic relocation. A medial patellar retinaculum lesion was noted. The patient recovered fully after splinting, bracing, and physiotherapy.  

Correct ICD-10-AM category:  
S83 - Dislocation and sprain of joints and ligaments of knee.  

Model output:  
The model generated “A87.5”, reduced to A87, which corresponds to viral meningitis.  

Case #5:  
Clinical summary:
A 56-year-old man presented with acute right lower extremity pain, redness, swelling, and fever. He was diagnosed with cellulitis of the right leg and required intravenous antibiotics, bedside and operative debridement, negative-pressure wound therapy, and later skin grafting. The wound progressively healed, and the patient was discharged with follow-up arranged.

Correct ICD-10-AM category:  
L03 - Cellulitis.  

Model output:  
The model produced “C078.1”, reduced to C07, corresponding to a secondary malignant neoplasm of the respiratory and digestive organs.  



The second prompt provides more explicit decision guidance by clarifying how to choose the principal diagnosis when multiple conditions are present. It also instructs the model to select the most defensible diagnosis when documentation is ambiguous. Overall, it includes more detailed reasoning constraints while maintaining the same strict output format.


In [ ]:
zero_shot_prompt_v2 = """
    You are an expert Australian clinical coder.

    Your task:
    Determine the PRINCIPAL DIAGNOSIS from the hospital course summary, using ICD-10-AM coding rules (ACS 0001).
    Select the single condition that, after study, is chiefly responsible for the admission.

    Output rules:
    - Output EXACTLY ONE code.
    - Output ONLY the ICD-10-AM CATEGORY (first 3 characters: one letter + two digits).
    - No explanations, no text, no punctuation, no justification.
    - If multiple diagnoses are present, choose the principal diagnosis per ACS 0001.
    - If the note is unclear, select the *most defensible* principal diagnosis; NEVER output more than one code.

    Hospital course summary:
    {note_text}

    Output ONLY the ICD-10-AM CATEGORY code:

    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt_v2.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I0001
ICD-10-AM category: I00
Entry 1:
raw model output: **ICD-10-AM
ICD-10-AM category: UNK
Entry 2:
raw model output: **ICD-10-AM
ICD-10-AM category: UNK
Entry 3:
raw model output: Lateral Patellar Dislocation
ICD-10-AM category: UNK
Entry 4:
raw model output: C078
ICD-10-AM category: C07


Case #1: Rheumatic fever without mention of heart involvement    
Case #2,3,4 -  Error    
Case #5: Secondary malignant neoplasm of the respiratory and digestive organs.  

Let's use LLM and create a more strict version of previous prompt:

In [ ]:
zero_shot_prompt_v3 = """
    You are an expert Australian clinical coder with deep knowledge of ICD-10-AM.

    Hospital course summary:
    {note_text}

    Your task:
    Identify the PRINCIPAL DIAGNOSIS according to ICD-10-AM rules.

    Specific instructions:
    - Output EXACTLY ONE ICD-10-AM CATEGORY code (3 characters: one letter + two digits).
    - Output ONLY the code—no words, no punctuation, no explanation.
    - If the correct code cannot be confidently determined from the documentation, output: I don't know
    - Never guess or invent clinical information not stated in the summary.

    Now output ONLY the ICD-10-AM CATEGORY code (or “I don't know”):

    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt_v3.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: Meningococcal Meningitis
ICD-10-AM category: UNK
Entry 1:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 2:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 3:
raw model output: Lateral Patellar Dislocation
ICD-10-AM category: UNK
Entry 4:
raw model output: C078
ICD-10-AM category: C07


Model performance declines in following instructions, but it still correctly identified two diagnosis outputs.

Add more instructions:

In [ ]:
zero_shot_prompt_v4 = """
    You are an expert Australian clinical coder who strictly applies ICD-10-AM and ACS 0001 standards.
    Your reasoning must rely ONLY on the documented clinical facts in the hospital course summary.

    Hospital course summary:
    {note_text}

    Your task:
    Identify the PRINCIPAL DIAGNOSIS — the condition established after study to be chiefly responsible for the admission (ACS 0001).

    Strict output rules:
    - Output EXACTLY ONE ICD-10-AM CATEGORY code (format: one letter + two digits, e.g., I21).
    - Output MUST match the regex: ^[A-Z][0-9]{{2}}$
    - Do NOT output subcategories, decimals, diagnosis names, or multiple codes.
    - Do NOT provide explanations, clarifications, or commentary.
    - If the documentation does NOT provide enough clear evidence to confidently determine a principal diagnosis, output exactly: I don't know
    - Never guess, infer undocumented conditions, or hallucinate clinical details.

    Tone:
    - Extremely concise
    - Neutral and factual
    - No additional text of any kind

    Now output ONLY the ICD-10-AM CATEGORY code (or “I don't know”):
    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt_v4.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I21
ICD-10-AM category: I21
Entry 1:
raw model output: I21
ICD-10-AM category: I21
Entry 2:
raw model output: I21
ICD-10-AM category: I21
Entry 3:
raw model output: I21
ICD-10-AM category: I21
Entry 4:
raw model output: I21
ICD-10-AM category: I21


It seems like the model only uses the code from the prompt.

In [ ]:
zero_shot_prompt_v5 = """
    You are an expert Australian clinical coder who strictly applies ICD-10-AM and ACS 0001 standards.
    Your reasoning must rely ONLY on the documented clinical facts in the hospital course summary.

    Hospital course summary:
    {note_text}

    Your task:
    Identify the PRINCIPAL DIAGNOSIS — the condition established after study to be chiefly responsible for the admission.

    Strict output rules:
    - Output EXACTLY ONE ICD-10-AM CATEGORY code (format: one letter + two digits).
    - Output MUST match the regex: ^[A-Z][0-9]{{2}}$
    - Do NOT output subcategories, decimals, diagnosis names, or multiple codes.
    - Do NOT provide explanations, clarifications, or commentary.
    - If the documentation does NOT provide enough clear evidence to confidently determine a principal diagnosis, output exactly: I don't know
    - Never guess, infer undocumented conditions, or hallucinate clinical details.

    Tone:
    - Extremely concise
    - Neutral and factual
    - No additional text of any kind

    Now output ONLY the ICD-10-AM CATEGORY code (or “I don't know”):
    """

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt_v5.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 1:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 2:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 3:
raw model output: Lateral Patellar Dislocation
ICD-10-AM category: UNK
Entry 4:
raw model output: I don't know
ICD-10-AM category: UNK


Long prompts are detrimental for small model performance. Let's try more concise approach.

In [ ]:
one_shot_prompt_v1 = """
    You are an expert Australian clinical coder.

    Example:
    Hospital course summary:
    "Patient admitted with chest pain; after study diagnosed with acute myocardial infarction."
    ICD-10-AM code:
    I21

    Now code this case:
    "{note_text}"

    Rules:
    - Output EXACTLY ONE ICD-10-AM CATEGORY (1 letter + 2 digits).
    - No words, no punctuation, no explanations.
    - If unclear, output: I don't know

    Code:

"""


In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = one_shot_prompt_v1.format(note_text=note_text) # inject the current note into the prompt template
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I2108.1
ICD-10-AM category: I21
Entry 1:
raw model output: I2100
ICD-10-AM category: I21
Entry 2:
raw model output: I2100
ICD-10-AM category: I21
Entry 3:
raw model output: I210500
ICD-10-AM category: I21
Entry 4:
raw model output: I2100
ICD-10-AM category: I21


The same problem. It seems like if the model sees any code, it sticks with it.

In [ ]:
three_shot_prompt_v1 = """
    You are an expert Australian clinical coder.

Examples:

1)
Summary: "Chest pain; after study = acute myocardial infarction."
Code: I21

2)
Summary: "Fever, cough, hypoxia; CXR = pneumonia."
Code: J18

3)
Summary: "Polyuria, polydipsia, high glucose; dx = type 2 diabetes."
Code: E11

Now code this case:
{note_text}

Rules:
- Output ONE ICD-10-AM CATEGORY (1 letter + 2 digits)
- No words, no punctuation, no explanations
- If unclear, output: I don't know

Code:

"""

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = three_shot_prompt_v1.format(note_text=note_text)
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: I21
ICD-10-AM category: I21
Entry 1:
raw model output: I21
ICD-10-AM category: I21
Entry 2:
raw model output: I21
ICD-10-AM category: I21
Entry 3:
raw model output: I21
ICD-10-AM category: I21
Entry 4:
raw model output: I21
ICD-10-AM category: I21


Maybe we need a shorter version.

In [ ]:
one_shot_prompt_v2 = """
    You are an expert Australian clinical coder.

    Example:
    Hospital course summary:
    "Patient admitted with chest pain; after study diagnosed with acute myocardial infarction."
    ICD-10-AM code:
    one letter + two digits

    Now summarise this case and code it:
    "{note_text}"

    Rules:
    - Output EXACTLY ONE ICD-10-AM CATEGORY (1 letter + 2 digits).
    - No words, no punctuation, no explanations.
    - If unclear, output: I don't know

    Code:

"""

In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = one_shot_prompt_v2.format(note_text=note_text)
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: ICD-10-AM Category
ICD-10-AM category: UNK
Entry 1:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 2:
raw model output: ICD-10-AM category
ICD-10-AM category: UNK
Entry 3:
raw model output: I don't know
ICD-10-AM category: UNK
Entry 4:
raw model output: ICD-10-AM category
ICD-10-AM category: UNK


The very first prompt performed the best.

In [ ]:
zero_shot_prompt_v6 = """
    You are an expert Australian clinical coder.
    Identify the PRINCIPAL DIAGNOSIS from the hospital course summary and output its ICD-10-AM CATEGORY code.
    Rules:
    - Output EXACTLY ONE code.
    - Output ONLY the 3-character ICD-10-AM CATEGORY (1 letter + 2 digits).
    - No extra words, punctuation, or explanation.
    Hospital course summary:
    {note_text}
    Now output ONLY the ICD-10-AM CATEGORY code:
    """


In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = zero_shot_prompt_v6.format(note_text=note_text)
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Entry 0:
raw model output: B85.1
ICD-10-AM category: B85
Entry 1:
raw model output: C628
ICD-10-AM category: C62
Entry 2:
raw model output: C045.1
ICD-10-AM category: C04
Entry 3:
raw model output: A85.4
ICD-10-AM category: A85
Entry 4:
raw model output: C078.1
ICD-10-AM category: C07


It outputs code, at least. But they are still incorrect.

In [ ]:
self_check_prompt = """
    You are an expert Australian clinical coder.

    Hospital course summary:
    {note_text}

    First, silently check:
    - What condition was chiefly responsible for the admission?
    - Is it explicitly documented?
    - Does it correspond to exactly one ICD-10-AM CATEGORY code?

    Now output ONLY the 3-character ICD-10-AM code (1 letter + 2 digits), with no explanation:
    """





In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = self_check_prompt.format(note_text=note_text)
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: MENINGITIS
ICD-10-AM category: UNK
Entry 1:
raw model output: A 71-year-old man with
ICD-10-AM category: UNK
Entry 2:
raw model output: 111111
ICD-10-AM category: UNK
Entry 3:
raw model output: A84.50
ICD-10-AM category: A84
Entry 4:
raw model output: C078
ICD-10-AM category: C07


<img src = 'https://drive.google.com/uc?id=1V8FBXSkNHntL8VOq5lgwL5wHyDy1iT6U' width = 600 alt='meningitis'>

"I have no mouth, and I must scream"

In [ ]:
cot_hidden_prompt = """
    You are an expert Australian clinical coder.

    Hospital course summary:
    {note_text}

    Let’s think step by step about which condition is the PRINCIPAL DIAGNOSIS and what its ICD-10-AM CATEGORY code is.

    After thinking, output ONLY the 3-character ICD-10-AM CATEGORY code (1 letter + 2 digits), with no words, no punctuation, no explanation.
"""


In [ ]:
codes = []
for i in range(5):
    note_text = dev_set[i]["note"]
    prompt = cot_hidden_prompt.format(note_text=note_text)
    raw, code = run_prompt(prompt)
    print(f"Entry {i}:")
    print("raw model output:", raw)
    print("ICD-10-AM category:", code)
    codes.append(code)

Entry 0:
raw model output: MENINGITIS
ICD-10-AM category: UNK
Entry 1:
raw model output: SPHONYCITIS
ICD-10-AM category: UNK
Entry 2:
raw model output: CPR
ICD-10-AM category: UNK
Entry 3:
raw model output: A
ICD-10-AM category: UNK
Entry 4:
raw model output: C078
ICD-10-AM category: C07


<img src = 'https://drive.google.com/uc?id=12oridlOzqC0pPkTPFOgH72brye5sz86t' width = 600 alt='sphonycitis'>    

I know, little bro, I know.
It is time to stop.



---



---



---



© 2025 Copyright The University of New South Wales - CRICOS 00098G